In [88]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
import re
import plotly.express as px
import config
from fredapi import Fred
import os
import shutil
from fpdf import FPDF

# User will need a FRED API key, can sign up for one here https://fred.stlouisfed.org/docs/api/api_key.html
# replace with your own API key: fred = Fred(api_key='YOUR KEY HERE')
fred = Fred(api_key=config.api_key)

In [89]:
from datetime import datetime, timedelta, date

class plot_obj:
  def __init__(self, df, title, units, notes, years_back, height = 400, width = 600):
    
    end_date_plot = df.index.max()
    start_date_plot = end_date_plot - timedelta(days = (365*years_back))
    self.df = df.loc[start_date_plot:end_date_plot]
    
    self.title = title
    self.units = units
    self.notes = notes
    self.height = height
    self.width = width
    
  
  def plot_clean(self):
    colnames = self.df.columns
    fig = px.line(self.df, x=self.df.index, y=colnames, title=self.title, template="plotly_white", width = self.width, height = self.height )
    fig.update_layout(
        
        font_family="Georgia",
        font_color="black",
        yaxis_title = self.units,
        xaxis_title = "Date"
    )
    fig.update_traces(connectgaps=True)
    fig.update_layout(showlegend=False)
    return fig 
    



In [91]:
#Define a function to retrieve series, store as objects, and plot data 
def fredget(varlist,start_date, end_date, plots=True,freq="",name_map = dict()):
    df_collect = pd.DataFrame()
    plot_obs = list()
    idx = 0
    
    PLOT_DIR = 'plots/fredget'
    try:
        shutil.rmtree(PLOT_DIR)
        os.mkdir(PLOT_DIR)
    except FileNotFoundError:
        os.mkdir(PLOT_DIR)
        
    for var in varlist:
        #series_add = fred.get_series(var, observation_start=s_d, observation_end=e_d,frequency ='m')
        df_add = fred.get_series(var, observation_start = start_date, observation_end=end_date, frequency =freq).to_frame(name = var)
        info_add = fred.get_series_info(var).to_frame(name = var)
        
        if var in name_map:
            info_add.loc['title', var] = name_map[var]
            
        plot_obs.append(plot_obj(df_add, info_add.loc['title', var], info_add.loc['units', var] ,"notes",5))
        
        print(var)
        if plots == True:
            #df_add.plot(y=var, use_index=True, kind='line', title = info_add.loc['title', var], ylabel = info_add.loc['units', var] )
            fig = plot_obs[idx].plot_clean()
            fig.show()
            fig.write_image(f"{PLOT_DIR}fig{idx}.png")

            
        idx = idx + 1 
        if df_collect.empty == True:
            df_collect = df_add
            info_store = info_add
        else:
            df_collect = pd.merge(df_collect, df_add, left_index=True, right_index=True, how='outer')
            info_store = pd.merge(info_store, info_add, left_index=True, right_index=True, how='left')
            
    df_collect.sort_index()
    return df_collect,info_store



#############################################
#Set Options Here:
start_d ='1982-01-01'
end_d='2023-02-03'
freq = "m" #THIS IS OPTIONAL, can leave blank
#List codes to search for 
FRED_list = ['T10Y2Y','MORTGAGE30US','EFFR','SP500','WALCL','BAMLH0A0HYM2']

Name_remap = {'T10Y2Y': '10-Year Minus 2-Year Treasury', 'WALCL': 'Reserve Balances: Total Assets'}
#############################################

#Optional inputs: plots = True
df,dfinfo = fredget(FRED_list,start_d,end_d,name_map = Name_remap)

T10Y2Y


ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido
